In [1]:
import cv2
from datetime import datetime
import time
import pandas as pd

# Initialize the webcam (0 is the default camera)
timeBeforeCapDefined = datetime.now() 
cap = cv2.VideoCapture(1)
timeAfterCapDefined = datetime.now() 

cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

initalFrameReadStart = datetime.now()
ret, frame = cap.read()
initalFrameReadEnd = datetime.now()
ret, frame = cap.read()


In [2]:
timeAfterCapDefined - timeBeforeCapDefined

datetime.timedelta(microseconds=266663)

In [3]:
initalFrameReadEnd - initalFrameReadStart

datetime.timedelta(seconds=1, microseconds=188891)

In [4]:
import torch.multiprocessing as mp
from modelWorker import model_worker
mp.set_start_method('spawn', force=True)

model_input_queue = mp.Queue()
model_output_queue = mp.Queue()

# Start the worker process
model_process = mp.Process(target=model_worker, args=(model_input_queue, model_output_queue))
model_process.start()

# time.sleep(1)
print(model_process.is_alive())
model_input_queue.put(frame)
result = model_output_queue.get()
print("Model output:", result)


True
waiting for frame!
Loading yolo11n-pose.torchscript for TorchScript inference...

0: 640x640 1 person, 5.6ms
Speed: 3.8ms preprocess, 5.6ms inference, 174.9ms postprocess per image at shape (1, 3, 640, 640)
waiting for frame!
Model output: True


In [5]:
from multiprocessing import Process, Queue
from writerWorker import writer_worker

writer_input_queue = Queue()
writer_output_queue = Queue()

writer_process = Process(target=writer_worker, args=(writer_input_queue, writer_output_queue))
writer_process.start()

no device name set


no device name set
in writer worker


In [6]:
import time
import tzlocal

frameCount = 0


# wait till a round 15 seconds
currTime = datetime.now()
time.sleep((14 - (currTime.second % 15)) + (1 - currTime.microsecond/1_000_000))

readTimes = [datetime.now(tzlocal.get_localzone())]
ret, frame = cap.read()
mybuffer = [frame]
model_input_queue.put(frame)

def modelResult():
    if model_output_queue.empty(): print("waiting for processing")
    return model_output_queue.get()

def delayTill100ms():
    msToDelay = 100 - ((datetime.now().microsecond / 1000) % 100)
    time.sleep(msToDelay/1000)

while True:
    delayTill100ms()
    
    #logging and frame cap
    readTimes.append(datetime.now(tzlocal.get_localzone()))
    ret, frame = cap.read()
    mybuffer.append(frame)

    if datetime.now().second % 15 == 0 and datetime.now().microsecond < 100_000:
        # if there was people then save the last 15 seconds
        if modelResult():
            writer_input_queue.put((readTimes, mybuffer))
        else:
            # else just save the one frame analyzed for a timelapse
            writer_input_queue.put((readTimes,[mybuffer[0]]))
        
        
        delayTill100ms()
        readTimes = [datetime.now(tzlocal.get_localzone())]
        ret, frame = cap.read()
        mybuffer = [frame]
        model_input_queue.put(frame)


    frameCount += 1


got frame!

0: 640x640 1 person, 128.3ms
Speed: 4.9ms preprocess, 128.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
got results!
waiting for frame!
got frame!

0: 640x640 1 person, 101.8ms
Speed: 6.6ms preprocess, 101.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
got results!
waiting for frame!
got frame!

0: 640x640 1 person, 85.3ms
Speed: 8.5ms preprocess, 85.3ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
got results!
waiting for frame!
got frame!

0: 640x640 1 person, 251.7ms
Speed: 10.1ms preprocess, 251.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
got results!
waiting for frame!
got frame!

0: 640x640 1 person, 147.3ms
Speed: 7.9ms preprocess, 147.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
got results!
waiting for frame!
got frame!

0: 640x640 1 person, 134.4ms
Speed: 18.1ms preprocess, 134.4ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
got re

KeyboardInterrupt: 

In [ ]:
# Release the webcam and close the output file
cap.release()
output.release()

In [5]:
metaDf['timeBetweenReads'] =  metaDf['preReadTime'].shift(-1) - metaDf['preReadTime']
(metaDf['timeBetweenReads'].dt.total_seconds() * 1000).describe()

count    199.000000
mean     100.000040
std        0.353543
min       96.995000
25%       99.996000
50%      100.000000
75%      100.005000
max      103.005000
Name: timeBetweenReads, dtype: float64

In [6]:
for e in metaDf['preReadTime'].tolist():
    print(e)

2024-11-01 08:59:07.700074
2024-11-01 08:59:07.800073
2024-11-01 08:59:07.900072
2024-11-01 08:59:08.000073
2024-11-01 08:59:08.100188
2024-11-01 08:59:08.200069
2024-11-01 08:59:08.300065
2024-11-01 08:59:08.400065
2024-11-01 08:59:08.500073
2024-11-01 08:59:08.600184
2024-11-01 08:59:08.700077
2024-11-01 08:59:08.800075
2024-11-01 08:59:08.900122
2024-11-01 08:59:09.000077
2024-11-01 08:59:09.100074
2024-11-01 08:59:09.200074
2024-11-01 08:59:09.300074
2024-11-01 08:59:09.400070
2024-11-01 08:59:09.500074
2024-11-01 08:59:09.600064
2024-11-01 08:59:09.700073
2024-11-01 08:59:09.800077
2024-11-01 08:59:09.900079
2024-11-01 08:59:10.000077
2024-11-01 08:59:10.100073
2024-11-01 08:59:10.200081
2024-11-01 08:59:10.300073
2024-11-01 08:59:10.400076
2024-11-01 08:59:10.500075
2024-11-01 08:59:10.600072
2024-11-01 08:59:10.700074
2024-11-01 08:59:10.800077
2024-11-01 08:59:10.900074
2024-11-01 08:59:11.000081
2024-11-01 08:59:11.100078
2024-11-01 08:59:11.200077
2024-11-01 08:59:11.300076
2